In [29]:
import numpy as np

# Create environment

In [30]:
number_of_states = 16
number_of_terminal_states = 2
number_of_non_terminal_states = number_of_states - number_of_terminal_states

In [31]:
max_number_of_actions = 4

In [32]:
number_of_actions_per_non_terminal_state = np.repeat(a = max_number_of_actions, repeats = number_of_non_terminal_states)

In [33]:
number_of_state_action_successor_states = np.repeat(a = 1, repeats = number_of_states * max_number_of_actions)

In [34]:
number_of_state_action_successor_states = np.reshape(a = number_of_state_action_successor_states, newshape = (number_of_states, max_number_of_actions))

In [35]:
state_action_successor_state_indices = np.array([1, 0, 14, 4, 2, 1, 0, 5, 2, 2, 1, 6, 4, 14, 3, 7, 5, 0, 3, 8, 6, 1, 4, 9, 6, 2, 5, 10, 8, 3, 7, 11, 9, 4, 7, 12, 10, 5, 8, 13, 10, 6, 9, 15, 12, 7, 11, 11, 13, 8, 11, 12, 15, 9, 12, 13], dtype = np.int64)

In [36]:
state_action_successor_state_transition_probabilities = np.repeat(a = 1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [37]:
state_action_successor_state_rewards = np.repeat(a = -1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [38]:
state_action_successor_state_indices = np.reshape(a = state_action_successor_state_indices, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_transition_probabilities = np.reshape(a = state_action_successor_state_transition_probabilities, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_rewards = np.reshape(a = state_action_successor_state_rewards, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))

# Set hyperparameters

In [39]:
#Set the n steps
n_steps = 4
# Set the number of episodes
number_of_episodes = 10000
# Set the maximum episode length
maximum_episode_length = 2000
# Set learning rate alpha
alpha = 0.1
# Set discounting factor gamma
discounting_factor_gamma = 1.0

# Create value function and policy arrays

In [40]:
# Create epsiode log
episode_log = {"state_index": np.repeat(a = -1, repeats = maximum_episode_length), 
               "action_index": np.repeat(a = -1, repeats = maximum_episode_length), 
               "reward": np.repeat(a = 0.0, repeats = maximum_episode_length), 
               "sigma": np.repeat(a = 0.0, repeats = maximum_episode_length), 
               "importance_sampling_ratio": np.repeat(a = 1.0, repeats = maximum_episode_length)}

In [41]:
state_action_value_function = np.repeat(a = 0.0, repeats = number_of_states * max_number_of_actions)
state_action_value_function = np.reshape(a = state_action_value_function, newshape = (number_of_states, max_number_of_actions))

In [42]:
target_policy = np.repeat(a = 1.0 / max_number_of_actions, repeats = number_of_non_terminal_states * max_number_of_actions)
target_policy = np.reshape(a = target_policy, newshape = (number_of_non_terminal_states, max_number_of_actions))

In [43]:
behavior_policy = np.repeat(a = 1.0 / max_number_of_actions, repeats = number_of_non_terminal_states * max_number_of_actions)
behavior_policy = np.reshape(a = behavior_policy, newshape = (number_of_non_terminal_states, max_number_of_actions))

# Create algorithm

In [44]:
# Set random seed so that everything is reproducible
np.random.seed(seed = 0)

In [45]:
# This function initializes episodes
def initialize_epsiode(number_of_non_terminal_states, max_number_of_actions, maximum_episode_length, behavior_policy, episode_log):
    # Initial state
    episode_log["state_index"][0] = np.random.randint(low = 0, high = number_of_non_terminal_states, dtype = np.int64) # randomly choose an initial state from all non-terminal states

    # Get initial action
    episode_log["action_index"][0] = np.random.choice(a = max_number_of_actions, p = behavior_policy[episode_log["state_index"][0], :])

    return maximum_episode_length, episode_log

In [46]:
# This function selects a policy greedily from the state-action-value function
def greedy_policy_from_state_action_function(state_action_value_function, state_index, policy):
    # Save max state-action value and find the number of actions that have the same max state-action value
    max_action_value = np.max(a = state_action_value_function[state_index, :])
    max_action_count = np.count_nonzero(a = state_action_value_function[state_index, :] == max_action_value)
    
    # Apportion policy probability across ties equally for state-action pairs that have the same value and zero otherwise
    max_policy_apportioned_probability_per_action = 1.0 / max_action_count
    policy[state_index, :] = np.where(state_action_value_function[state_index, :] == max_action_value, max_policy_apportioned_probability_per_action, 0.0)
    
    return policy

In [47]:
# This function loops through episodes and updates the policy
def loop_through_episode(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, target_policy, behavior_policy, alpha, discounting_factor_gamma, maximum_episode_length, max_timestep, episode_log, n_steps):
    # Loop through episode steps until termination
    for t in range(0, maximum_episode_length):
        # Spend a little memory to save computation time
        t_mod_n_plus_1 = t % (n_steps + 1);
        t_plus_1_mod_n_plus_1 = (t + 1) % (n_steps + 1);
        
        if t < max_timestep:
            # Get reward
            successor_state_transition_index = np.random.choice(a = number_of_state_action_successor_states[episode_log["state_index"][t_mod_n_plus_1], episode_log["action_index"][t_mod_n_plus_1]], p = state_action_successor_state_transition_probabilities[episode_log["state_index"][t_mod_n_plus_1], episode_log["action_index"][t_mod_n_plus_1], :])

            episode_log["reward"][t_plus_1_mod_n_plus_1] = state_action_successor_state_rewards[episode_log["state_index"][t_mod_n_plus_1], episode_log["action_index"][t_mod_n_plus_1], successor_state_transition_index]

            # Get next state
            episode_log["state_index"][t_plus_1_mod_n_plus_1] = state_action_successor_state_indices[episode_log["state_index"][t_mod_n_plus_1], episode_log["action_index"][t_mod_n_plus_1], successor_state_transition_index]

            # Check to see if we actioned into a terminal state
            if episode_log["state_index"][t_plus_1_mod_n_plus_1] >= number_of_non_terminal_states:
                max_timestep = t + 1
            else:
                # Get next action
                episode_log["action_index"][t_plus_1_mod_n_plus_1] = np.random.choice(a = max_number_of_actions, p = behavior_policy[episode_log["state_index"][t_plus_1_mod_n_plus_1], :])
                
                # Get next sigma
                episode_log["sigma"][t_plus_1_mod_n_plus_1] = np.random.choice(a = [0.0, 1.0])

                # Get next importance sampling ratio
                episode_log["importance_sampling_ratio"][t_plus_1_mod_n_plus_1] = target_policy[episode_log["state_index"][t_plus_1_mod_n_plus_1], episode_log["action_index"][t_plus_1_mod_n_plus_1]] / behavior_policy[episode_log["state_index"][t_plus_1_mod_n_plus_1], episode_log["action_index"][t_plus_1_mod_n_plus_1]] 
                
        tau = t - n_steps + 1 # tau is the time whose estimate is being updated
        
        if tau >= 0:
            expected_return = 0.0
            
            for k in range(np.min(a = [t + 1, max_timestep]), tau, -1):
                if k == max_timestep:
                    # Calculate expected return
                    expected_return = episode_log["reward"][max_timestep % (n_steps + 1)]
                else:
                    # Spend a little memory to save computation time
                    k_mod_n_plus_1 = k % (n_steps + 1);

                    # Calculate expected state value function from target policy
                    state_value_function_expected_value_on_policy = np.sum(a = target_policy[episode_log["state_index"][k_mod_n_plus_1], :] * state_action_value_function[episode_log["state_index"][k_mod_n_plus_1], :])

                    # Calculate expected return
                    expected_return = episode_log["reward"][k_mod_n_plus_1] + discounting_factor_gamma * ((episode_log["sigma"][k_mod_n_plus_1] * episode_log["importance_sampling_ratio"][k_mod_n_plus_1] + (1.0 - episode_log["sigma"][k_mod_n_plus_1]) * target_policy[episode_log["state_index"][k_mod_n_plus_1], episode_log["action_index"][k_mod_n_plus_1]]) * (expected_return - state_action_value_function[episode_log["state_index"][k_mod_n_plus_1], episode_log["action_index"][k_mod_n_plus_1]]) + state_value_function_expected_value_on_policy);
            
            # Spend a little memory to save computation time
            tau_mod_n_plus_1 = tau % (n_steps + 1)

            # Calculate state-action-function at tau timestep
            state_action_value_function[episode_log["state_index"][tau_mod_n_plus_1], episode_log["action_index"][tau_mod_n_plus_1]] += alpha * (expected_return - state_action_value_function[episode_log["state_index"][tau_mod_n_plus_1], episode_log["action_index"][tau_mod_n_plus_1]])

            # Choose policy for chosen state by epsilon-greedy choosing from the state-action-value function
            target_policy = greedy_policy_from_state_action_function(state_action_value_function, episode_log["state_index"][tau_mod_n_plus_1], target_policy)

        if tau == max_timestep - 1:
            break # break episode step loop, move on to next episode

    return state_action_value_function, target_policy

In [48]:
def off_policy_n_step_bootstrapping_q_sigma(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, target_policy, behavior_policy, alpha, discounting_factor_gamma, maximum_episode_length, episode_log, n_steps):
    for episode in range(0, number_of_episodes):
        # Initialize episode to get initial state and action
        max_timestep, episode_log = initialize_epsiode(number_of_non_terminal_states, max_number_of_actions, maximum_episode_length, behavior_policy, episode_log)

        # Loop through episode and update the policy
        state_action_value_function, target_policy = loop_through_episode(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, target_policy, behavior_policy, alpha, discounting_factor_gamma, maximum_episode_length, max_timestep, episode_log, n_steps)
    
    return state_action_value_function, target_policy

# Run algorithm

In [49]:
# Print initial arrays
print("\nInitial state-action value function")
print(state_action_value_function)

print("\nInitial target policy")
print(target_policy)

# Run off policy n-step bootstrapping Q(sigma)
state_action_value_function, target_policy = off_policy_n_step_bootstrapping_q_sigma(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, target_policy, behavior_policy, alpha, discounting_factor_gamma, maximum_episode_length, episode_log, n_steps)

# Print final results
print("\nFinal state-action value function")
print(state_action_value_function)

print("\nFinal target policy")
print(target_policy)


Initial state-action value function
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial target policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Final state-action value function
[[-3. -2. -1. -3.]
 [-4. -3. -2. -4.]
 [-4. -4. -3. -3.]
 [-3. -1. -2. -3.]
 [-4. -2. -2. -4.]
 [-3. -3. -3. -3.]
 [-3. -4. -4. -2.]
 [-4. -2. -3. -4.]
 [-3. -3. -3. -3.]
 [-2. -4. -4. -2.]
 [-2. -3. -3. -1.]
 [-3. -3. -4. -4.]
 [-2. -4. -4. -3.]
 [-1. -3. -3. -2.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Final target policy
[[0.  0.  1.  0